In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_727751/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
8,model_3_6_23,0.814739,0.571284,0.628675,0.634006,22.342472,60.300930,55.305943,57.950352,"Hidden Size=[30, 15], regularizer=0.02, learni..."
9,model_3_6_22,0.813156,0.564401,0.635134,0.633628,22.533434,61.269150,54.343941,58.010273,"Hidden Size=[30, 15], regularizer=0.02, learni..."
11,model_3_6_24,0.811209,0.576442,0.595021,0.621534,22.768221,59.575527,60.318535,59.925205,"Hidden Size=[30, 15], regularizer=0.02, learni..."
15,model_3_7_12,0.800239,0.563913,0.672639,0.674295,24.091280,28.892586,62.255650,44.592850,"Hidden Size=[30, 15], regularizer=0.02, learni..."
16,model_3_7_13,0.797437,0.546146,0.651740,0.656082,24.429117,30.069723,66.230141,47.086395,"Hidden Size=[30, 15], regularizer=0.02, learni..."
17,model_3_7_14,0.794668,0.529150,0.631409,0.638439,24.763088,31.195766,70.096542,49.502018,"Hidden Size=[30, 15], regularizer=0.02, learni..."
18,model_3_7_15,0.791796,0.512503,0.611037,0.620857,25.109430,32.298702,73.970833,51.909081,"Hidden Size=[30, 15], regularizer=0.02, learni..."
19,model_3_7_16,0.788705,0.495962,0.590522,0.603210,25.482208,33.394638,77.872147,54.325214,"Hidden Size=[30, 15], regularizer=0.02, learni..."
20,model_3_7_17,0.785687,0.481136,0.571712,0.587116,25.846249,34.376930,81.449432,56.528709,"Hidden Size=[30, 15], regularizer=0.02, learni..."
25,model_3_8_5,0.765606,0.611586,0.703562,0.652176,28.268002,90.868233,24.308880,59.546154,"Hidden Size=[30, 15], regularizer=0.02, learni..."
